### Import

In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from spotipy import Spotify
from spotipy.exceptions import SpotifyException

import base64
import requests
from requests import post, get
import time

In [2]:
# Preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import KNNImputer, SimpleImputer

# Feature Selection + Dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance

# Models (estimators)
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.cluster import KMeans

# Evaluate Models
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict
from sklearn.model_selection import learning_curve, StratifiedKFold
## Scoring + Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, max_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import ConfusionMatrixDisplay, PrecisionRecallDisplay #.from_estimator()
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import classification_report

# Pipelines
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression, make_classification, make_multilabel_classification, make_blobs
import pickle

# Dataset

In [132]:
df = pd.read_csv('/home/mellerio/code/moogrocker/HarmonIQ/data/dataset.csv')

In [133]:
df = df.drop(columns=['Unnamed: 0'])

In [134]:
df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [8]:
df_gpt = df.describe()

In [156]:
df_million = pd.read_csv('../../raw_data/tracks_features.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../../raw_data/tracks_features.csv'

# Spotify API

## __init__ & Credential

In [3]:
# scope = "user-library-modify playlist-read-private user-library-read playlist-modify-private"

In [4]:
client_id = '4fe38c597d7c41d2a418144a87d7d082'
client_secret = '7deaab2f08154a97905868d89919fc55'
redirect_uri = 'https://localhost:8888/callback/'
scope = "playlist-modify-private user-library-read playlist-read-private"
# scope = "user-library-read playlist-read-private playlist-modify-private"

In [5]:
sp = Spotify(auth_manager=SpotifyOAuth(
                 client_id=client_id,
                 client_secret=client_secret,
                 redirect_uri=redirect_uri,
                 scope=scope))

In [6]:
# auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(auth_manager=auth_manager)

In [7]:
# # Combine and encode
# credentials = f"{client_id}:{client_secret}"
# encoded_credentials = base64.b64encode(credentials.encode()).decode()

# print(f"Encoded Credentials: {encoded_credentials}")

# # Spotify token endpoint
# url = "https://accounts.spotify.com/api/token"

# # Request headers and body
# headers = {
#     "Authorization": f"Basic {encoded_credentials}"
# }
# data = {
#     "grant_type": "client_credentials"
# }

# # Make the request
# response = requests.post(url, headers=headers, data=data)

# if response.status_code == 200:
#     token = response.json()['access_token']
#     print(f"Access Token: {token}")
# else:
#     print(f"Failed to get token: {response.status_code}, {response.text}")

In [66]:
# authorization_token = f"Bearer {token}"
# url = "https://api.spotify.com/v1/me"
# headers = {"Authorization" : authorization_token}

# response = requests.get(url, headers=headers)
# user_data = response.json()
# # print(f"Spotify User ID: {user_data['id']}")
# user_data

In [41]:
# sp = spotipy.Spotify(auth=token)

In [80]:
# country_codes =  ['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL',
#                   'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI',
#                   'FR', 'DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE',
#                   'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC',
#                   'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT',
#                   'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']

# limit = 50
# language_code = 'EN'

## Data fetching

In [12]:
def get_playlist_categories(country_code, language_code, limit, offset=0):
    categories = []
    raw_categories = sp.categories(country=country_code, locale=language_code, limit=limit, offset=offset)
    for cat in raw_categories['categories']['items']:
        categories.append({'category':cat['name'], 'cat_id':cat['id']})
    return categories

In [ ]:
def get_playlists(category, cat_id, limit):

    # Get playlists from categories
    cat_id = "dinner"
    cat_playlists = sp.category_playlists(category_id=cat_id, limit=limit)
    for playlist in cat_playlists['playlists']['items']:
        # Create a temporary DataFrame for the current playlist
        temp_df = pd.DataFrame([{
            'id': playlist['id'],
            'name': playlist['name'],
            'description': playlist['description'],
            'category': category
        }])

        # Concatenate the temporary DataFrame with the main DataFrame
        playlists = pd.concat([playlists, temp_df], ignore_index=True)

    return playlists

In [ ]:
def get_featured_playlists(country_code, limit, locale):

    # Get featured playlists
    featured_playlists = sp.featured_playlists(locale=locale, country=country_code, limit=limit)
    for playlist in featured_playlists['playlists']['items']:
        # Create a temporary DataFrame for the current playlist
        temp_df = pd.DataFrame([{
            'id': playlist['id'],
            'name': playlist['name'],
            'description': playlist['description'],
            'category': 'featured_playlist'
        }])

        # Concatenate the temporary DataFrame with the main DataFrame
        playlists = pd.concat([playlists, temp_df], ignore_index=True)

    return playlists

In [131]:
def get_playlist_tracks(playlist_id):
    track_list = []
    playlists_tracks = sp.playlist_tracks(playlist_id=playlist_id)
    for track in playlists_tracks['items']:
        track_list.append(track['track']['id'])
    return track_list

In [132]:
def get_playlist_audio_feat(track_list, playlist_id):
    features = pd.DataFrame(sp.audio_features(track_list))
    features.drop(columns=['track_href', 'analysis_url', 'uri', 'type'])
    return features

## Data fetching bis

In [172]:
def get_user_profile():
    user_profile = sp.me()
    return user_profile['id']

In [173]:
get_user_profile()

'adauchez08'

In [174]:
sp.user('nickroberts')

{'display_name': 'Nick Roberts',
 'external_urls': {'spotify': 'https://open.spotify.com/user/nickroberts'},
 'followers': {'href': None, 'total': 16},
 'href': 'https://api.spotify.com/v1/users/nickroberts',
 'id': 'nickroberts',
 'images': [{'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=10151961536807142&height=300&width=300&ext=1735477299&hash=Aba2-I2ksQ8ljGo2A20sRLWy',
   'height': 300,
   'width': 300},
  {'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=10151961536807142&height=50&width=50&ext=1735477299&hash=AbaQRRgXEOeJ3NOnmF3icOCI',
   'height': 64,
   'width': 64}],
 'type': 'user',
 'uri': 'spotify:user:nickroberts'}

In [175]:
saved_tracks = sp.current_user_saved_tracks(limit=20, offset=0, market=None)

In [176]:
len(saved_tracks['items'])

8

In [177]:
nick_playlists = sp.user_playlists('113821586')

In [178]:
# me_playlists = sp.user_playlists('113821586')
playlist_names = []
playlist_ids = []

for playlist in nick_playlists['items']:
    playlist_names.append(playlist['name'])
    playlist_ids.append(playlist['id'])

playlist_names = pd.Series(playlist_names, name="playlist_names")
playlist_ids = pd.Series(playlist_ids, name="playlist_ids")
user_playlists_df = pd.DataFrame({playlist_names.name: playlist_names, playlist_ids.name: playlist_ids})

In [103]:
user_playlists_df.head()

,playlist_names,playlist_ids
0,Top Hits of the 2010s (2),5cfMUrQ3nXlTZP7JMCqzt7
1,2010-2020 Top Hits (2010’s) (2),6IAYwQ24xn2CD06OjjMkjl
2,Coding Music UI,3lvjpzXGx67e8QrPFYzJEd
3,Chutzpah,1HlH4GymkAftVj1IYqzXFI
4,2024- Change v7.1 nr,70CY7pCmtwBo60Ax66UOSz


In [37]:
playlist_tracks = sp.playlist('16BdYan33CxHYdQ2lmujvw')['tracks']['items']

In [38]:
len(playlist_tracks)

39

In [51]:
for i in range(len(playlist_tracks)):
    print(playlist_tracks[i]['track']['uri'], playlist_tracks[i]['track']['id'])
    # print(playlist_tracks[i]['track']['id'])

spotify:track:1LpxY3sL5V2BDwsoILCCFs 1LpxY3sL5V2BDwsoILCCFs
spotify:track:4FbetV1hiQIfUm5PgN3Q9D 4FbetV1hiQIfUm5PgN3Q9D
spotify:track:34iRFrc96fSVdPlwIwaDoK 34iRFrc96fSVdPlwIwaDoK
spotify:track:7poGa2k7wTlEzuML8pdbA2 7poGa2k7wTlEzuML8pdbA2
spotify:track:3gVll51uHNVhqm6lBg26pp 3gVll51uHNVhqm6lBg26pp
spotify:track:2Ov9dDRvhPHKieyVFam5TM 2Ov9dDRvhPHKieyVFam5TM
spotify:track:5WakY70QIvDzwxdJZ0TOMs 5WakY70QIvDzwxdJZ0TOMs
spotify:track:14ErDGNe7PzsPjJ21gzwwf 14ErDGNe7PzsPjJ21gzwwf
spotify:track:4OUkAm6SgbhK9YnVxFVYs8 4OUkAm6SgbhK9YnVxFVYs8
spotify:track:4miF4XSnRS0E7eXoTrzgmP 4miF4XSnRS0E7eXoTrzgmP
spotify:track:4nc2N1JCbktTbiahf8tNVw 4nc2N1JCbktTbiahf8tNVw
spotify:track:3aYmAoaPgGFm49ZOw2OQxK 3aYmAoaPgGFm49ZOw2OQxK
spotify:track:4vIboBMsVk0NziOuyEinkE 4vIboBMsVk0NziOuyEinkE
spotify:track:2BUL7sF7y5oBYUQarVCukR 2BUL7sF7y5oBYUQarVCukR
spotify:track:4ayNMLOAMbGmZiVquX8n6K 4ayNMLOAMbGmZiVquX8n6K
spotify:track:5QAxKBAZ6ohJ4eOUvBslCh 5QAxKBAZ6ohJ4eOUvBslCh
spotify:track:3PgPjbTS5XHweYHCMkpNG3 3Pg

In [115]:
tracks = sp.user_playlist_tracks(user='nick roberts', playlist_id='5cfMUrQ3nXlTZP7JMCqzt7')

In [136]:
tracks['items'][1]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3sgFRtyBnxXD5ESfmbK4dl'},
  'href': 'https://api.spotify.com/v1/artists/3sgFRtyBnxXD5ESfmbK4dl',
  'id': '3sgFRtyBnxXD5ESfmbK4dl',
  'name': 'LMFAO',
  'type': 'artist',
  'uri': 'spotify:artist:3sgFRtyBnxXD5ESfmbK4dl'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/2jLE4BoXHriQ96JagEtiDP'},
  'href': 'https://api.spotify.com/v1/artists/2jLE4BoXHriQ96JagEtiDP',
  'id': '2jLE4BoXHriQ96JagEtiDP',
  'name': 'Lauren Bennett',
  'type': 'artist',
  'uri': 'spotify:artist:2jLE4BoXHriQ96JagEtiDP'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/53sIBaVjXQhfH89Vu6nEGh'},
  'href': 'https://api.spotify.com/v1/artists/53sIBaVjXQhfH89Vu6nEGh',
  'id': '53sIBaVjXQhfH89Vu6nEGh',
  'name': 'GoonRock',
  'type': 'artist',
  'uri': 'spotify:artist:53sIBaVjXQhfH89Vu6nEGh'}]

In [165]:
new_playlist = sp.user_playlist_create('adauchez08', 'arthur_test1', public=False, collaborative=False, description='')

In [166]:
items_2_add = ['spotify:track:1LpxY3sL5V2BDwsoILCCFs', 'spotify:track:4FbetV1hiQIfUm5PgN3Q9D', 'spotify:track:3aYmAoaPgGFm49ZOw2OQxK']

In [167]:
new_playlist = sp.playlist_add_items(new_playlist['id'], items=items_2_add, position=None)

In [112]:
import pandas as pd
import spotipy
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '4fe38c597d7c41d2a418144a87d7d082'
client_secret = '7deaab2f08154a97905868d89919fc55'

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
def get_artist_genres(artists_lst:list):

    artist_genres = {}
    chunks = len(artists_lst) // 50 # 97800 unique artists
    last_chunks = len(artists_lst) % 50
    chunk_size = 50
    start = 0
    finish = chunk_size + 1

    for chunk in range(chunks):
        artists = sp.artists(artists_lst[start:finish])
        for artist in artists['artists']:
            artist_genres[artist['id']] = artist['genres'][0]
        print(f'chunk {chunck}/{chunks+last_chunks} done.')
        start = finish - 1(
        finish = finish + chunk_size

    if last_chunks > 0:
        for chunk in range(last_chunks):
            artists = sp.artists(artists_lst[len(artist_genres):(len(artists_lst)+1)])
            for artist in artists['artists']:
                artist_genres[artist['id']] = artist['genres'][0]
            print(f'chunk {chunks+chunk}/{chunks+last_chunks} done.')
            start = finish - 1
            finish = finish + chunk_size

    return artist_genres

In [137]:
artists_test = sp.artists(['3sgFRtyBnxXD5ESfmbK4dl', '2jLE4BoXHriQ96JagEtiDP', '53sIBaVjXQhfH89Vu6nEGh'])

In [143]:
artists_test['artists'][2]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/53sIBaVjXQhfH89Vu6nEGh'},
 'followers': {'href': None, 'total': 7521},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/53sIBaVjXQhfH89Vu6nEGh',
 'id': '53sIBaVjXQhfH89Vu6nEGh',
 'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b2737e8f3011d91e0b8970466085',
   'height': 640,
   'width': 640},
  {'url': 'https://i.scdn.co/image/ab67616d00001e027e8f3011d91e0b8970466085',
   'height': 300,
   'width': 300},
  {'url': 'https://i.scdn.co/image/ab67616d000048517e8f3011d91e0b8970466085',
   'height': 64,
   'width': 64}],
 'name': 'GoonRock',
 'popularity': 63,
 'type': 'artist',
 'uri': 'spotify:artist:53sIBaVjXQhfH89Vu6nEGh'}

## Preproc

In [ ]:
 def preprocess_data(data):
#     """Cleans and preprocesses the data."""
#     # Drop unnecessary columns
#     columns_to_drop = ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name']
#     data = data.drop(columns=columns_to_drop)

#     # Handle missing values
#     data = data.dropna()

#     # Separate features and target
#     X = data.drop(columns=['popularity'])
#     y = data['popularity']

#     # Convert `explicit` column to numeric
#     X['explicit'] = X['explicit'].astype(int)

#     # One-hot encode categorical features
#     categorical_features = ['track_genre']
#     numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_features)

#     # Preprocessing pipeline
#     preprocessor = ColumnTransformer(
#         transformers=[
#             ('num', StandardScaler(), numeric_features),
#             ('cat', OneHotEncoder(drop='first'), categorical_features)
#         ]
#     )

#     return X, y, preprocessor

# def split_data(X, y, test_size=0.2, random_state=42):
#     """Splits the data into training and testing sets."""
#     return train_test_split(X, y, test_size=test_size, random_state=random_state)


## Test 

In [8]:
track_info = sp.track('5SuOikwiRyPMVoIQDJUgSV')

In [13]:
track_info['external_ids']['isrc']

'JPVI02200563'

In [174]:
limit=50
language_code = 'EN'
country_code = 'US'

In [175]:
categories = get_playlist_categories(country_code=country_code, language_code=language_code, limit=limit, offset=50)
categories

[{'category': 'Blues', 'cat_id': '0JQ5DAqbMKFQiK2EHwyjcU'},
 {'category': 'Cooking & Dining', 'cat_id': '0JQ5DAqbMKFRY5ok2pxXJ0'},
 {'category': 'Travel', 'cat_id': '0JQ5DAqbMKFAQy4HL4XU2D'},
 {'category': 'Songwriters', 'cat_id': '0JQ5DAqbMKFSCjnQr8QZ3O'},
 {'category': 'Nature & Noise', 'cat_id': '0JQ5DAqbMKFI3pNLtYMD9S'},
 {'category': 'Funk & Disco', 'cat_id': '0JQ5DAqbMKFFsW9N8maB6z'},
 {'category': 'Latin', 'cat_id': '0JQ5DAqbMKFxXaXKP7zcDp'},
 {'category': 'Spotify Singles', 'cat_id': '0JQ5DAqbMKFDBgllo2cUIN'},
 {'category': 'Arab', 'cat_id': '0JQ5DAqbMKFQ1UFISXj59F'},
 {'category': 'Christian & Gospel', 'cat_id': '0JQ5DAqbMKFy0OenPG51Av'},
 {'category': 'Fresh Finds', 'cat_id': '0JQ5DAqbMKFImHYGo3eTSg'},
 {'category': 'Music + Talk', 'cat_id': '0JQ5DAqbMKFRNXsIvgZF9A'},
 {'category': 'Tastemakers', 'cat_id': '0JQ5DAqbMKFRKBHIxJ5hMm'}]

In [178]:
categories = get_playlist_categories(country_code=country_code, language_code=language_code, limit=limit)

for cat in categories:
    cat_playlists = get_playlists(category=cat['category'], cat_id=cat['cat_id'], limit=limit)

HTTP Error for GET to https://api.spotify.com/v1/browse/categories/dinner/playlists with Params: {'country': None, 'limit': 50, 'offset': 0} returned 404 due to Not Found


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/browse/categories/dinner/playlists?limit=50&offset=0:
 Not Found, reason: None

In [146]:
get_playlists(category='Mood', cat_id='0JQ5DAqbMKFzHmL4tf05da', limit=50)

HTTP Error for GET to https://api.spotify.com/v1/browse/categories/0JQ5DAqbMKFzHmL4tf05da/playlists with Params: {'country': None, 'limit': 50, 'offset': 0} returned 404 due to Not Found


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/browse/categories/0JQ5DAqbMKFzHmL4tf05da/playlists?limit=50&offset=0:
 Not Found, reason: None

In [170]:
sp.category_playlists(category_id='dinner', limit=50)

HTTP Error for GET to https://api.spotify.com/v1/browse/categories/dinner/playlists with Params: {'country': None, 'limit': 50, 'offset': 0} returned 404 due to Not Found


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/browse/categories/dinner/playlists?limit=50&offset=0:
 Not Found, reason: None

In [105]:
get_playlists(category='Mood', cat_id='0JQ5DAqbMKFzHmL4tf05da', limit=50)

HTTP Error for GET to https://api.spotify.com/v1/browse/categories/0JQ5DAqbMKFzHmL4tf05da/playlists with Params: {'country': 'US', 'limit': 50, 'offset': 0} returned 404 due to Not Found


NameError: name 'SpotifyException' is not defined

In [155]:
get_featured_playlists(locale= 'en_US', country_code=country_code, limit=limit)

HTTP Error for GET to https://api.spotify.com/v1/browse/featured-playlists with Params: {'locale': 'en_US', 'country': 'US', 'timestamp': None, 'limit': 50, 'offset': 0} returned 404 due to Not Found


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/browse/featured-playlists?locale=en_US&country=US&limit=50&offset=0:
 Not Found, reason: None

In [156]:
try:
    response = sp.featured_playlists(country='US')
    print(response)
except Exception as e:
    print(f"Error: {e}")

HTTP Error for GET to https://api.spotify.com/v1/browse/featured-playlists with Params: {'locale': None, 'country': 'US', 'timestamp': None, 'limit': 20, 'offset': 0} returned 404 due to Not Found


Error: http status: 404, code:-1 - https://api.spotify.com/v1/browse/featured-playlists?country=US&limit=20&offset=0:
 Not Found, reason: None


# MusicBrainz music API

## __init__ and credentials

In [35]:
! pip install ytmusicapi

In [36]:
from ytmusicapi import YTMusic

ytmusic = YTMusic()

In [ ]:
! ytmusicapi oauth

## New test

In [45]:
track_info = sp.track('6lfxq3CG4xtTiEg7opyCyx')

KeyboardInterrupt: 

In [ ]:
track_info['external_ids']['isrc']

In [17]:
df.shape[0]

114000

In [ ]:
df_isrc = []
chunks = 1
chunk_size = int(df.shape[0]/100)
start = 0
finish = chunk_size + 1

In [ ]:
for chunck in range(chunks):

    for index in range(start,finish):
        print('...')
        isrc = sp.track(df.iloc[index]['track_id'])['external_ids']['isrc']
        print(f'isrc {index} found.')
        df_isrc.append(isrc)

    start = start + chunk_size
    finish = finish + chunk_size

KeyboardInterrupt: 

In [ ]:
isrc = pd.Series(df_isrc)

In [32]:
df_isrc

['JPVI02200563',
 'QZHN32101189',
 'USVT34000002',
 'USNLR1800566',
 'USUM71615568',
 'TCADP1800198',
 'USSM11306713',
 'USEE10800667',
 'USEE10800806',
 'TCACF1515677',
 'PHUM72100313',
 'USEE11100768',
 'SE5IB1800630',
 'QZES81884962',
 'USUM71704507',
 'USRC10900386',
 'QM24S2006594',
 'QM24S2011689',
 'USEE10800806',
 'USM951500007',
 'USEE10800667',
 'USM951400001',
 'USSM11306713',
 'USEE11200150',
 'USAT21802784',
 'USEE10500841',
 'USEE10801739',
 'USEE10800814',
 'USEE10801739',
 'USEE10801739',
 'USEE10801739',
 'USEE10801739',
 'USUM71615938',
 'USAT21811525',
 'USAT21811525',
 'USAT21811525',
 'GBGLM0700092',
 'USAT22102164',
 'USAT22102157',
 'GBGLM0700092',
 'USUG12104377',
 'USAT22102161',
 'USAT22102156',
 'USAT22102156',
 'USAT22102767',
 'USAT22102156',
 'USAT22102156',
 'USAT22102156',
 'USAT22102156',
 'USAT22102156',
 'SE5IB1901528',
 'JPBV01021331',
 'US8D61300002',
 'USTC50842502',
 'UK42F1700157',
 'US8D61300008',
 'USKX21500005',
 'QZS7J2236602',
 'USSM10700068